In [5]:
import pydicom
import fnmatch
import os
import numpy as np
from pydicom.data import get_testdata_files
from pydicom.filereader import read_dicomdir
from scipy.spatial.distance import cdist
from os.path import dirname, join
from ipywidgets.widgets import * 
import ipywidgets as widgets
import cv2
import collections
from pprint import pprint
import matplotlib.pyplot as plt
import sys
import glob
import csv
import pandas as pd
from myfunctions import get_generated_mask,read_record,create_contour,create_mask,find_overlap_area


In [6]:
# setting---
num_img=1
thickness=1
BLUR = 21
CANNY_THRESH_1 = 20
CANNY_THRESH_2 = 150
DILATE_ITER = 1
ERODE_ITER = 1
size_dial,size_erod=1,1
max_diff=100
mask_accuracy=6
def click_save(event, x, y, flags, param):
#  grab references to the global variables
    global refPt
    if event == cv2.EVENT_LBUTTONDOWN:
        refPt.append((x, y))

In [9]:
# enter path to data directory
folder_data='/Users/Engin/Documents/Edwards/Projects/Echo/MS_test'


foldPath_split=folder_data.split('/')
result_fold='/'.join(foldPath_split[:-1])
# mask_path=os.path.join(result_fold,foldPath_split[-1]+'_'+'generated_mask')
mask_path=os.path.join(os.getcwd(),foldPath_split[-1]+'_'+'generated_mask')
help_path=os.path.join(os.getcwd(),'utility/help.png')
# current_rec=pd.read_csv('processes_records.csv')
path_to_records=[]
for path, subdirs, files in os.walk(folder_data):
    for name in files:
#         if name.startswith('IM'): 
        if fnmatch.fnmatch(name, '*IM*'):
            path_to_records.append([os.path.join(path, name)])
if not os.path.exists(mask_path):
    os.mkdir(mask_path)
#  start with current generated mask. Pass records that are already generated.
current_device=[]
available_model,available_folder,available_masks=get_generated_mask(mask_path)
print(available_folder)
help_img=cv2.imread(help_path)
size_h=help_img.shape
process_path=[]
q=0
for path_ in sorted(path_to_records):
    s=0
    if q==1:
        break
    path=path_[0]
    print(path)
    process_path.append(path)
    folder_name=path.split('/')[-3]
    file_name=path.split('/')[-1]
#     if folder_name not in available_folder:
    data,feats_rec,val=read_record(path,file_name,folder_name)
    if val:
        modelName='_'.join([feats_rec[1],feats_rec[0]])                               
        mask_fname='_'.join([feats_rec[1],feats_rec[0],folder_name,file_name+'.png'])
        mask_fpath=os.path.join(mask_path,mask_fname)
        print('Number of images in the record:{}'.format(data.shape[0]))
        cm=data.shape[-1]==3
        refPt=[]
        im_tst=data[num_img,:,:,:] if cm else data[num_img,:,:]
        img=im_tst.copy()
        img       
        print(modelName)
        if modelName in available_masks:            
            index1=0
            grayimg=cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) if cm else img
            acceptable_score=mask_accuracy*sum(grayimg.shape)/2
#             max_score = float('inf')
#             for Mask_n in available_masks[modelName]:
#                 s=evaluate_mask(Mask_n,grayimg,10)
#                   if s < max_score:
#                         max_score=s
            relative_masks=available_masks[modelName]
            max_score,_=find_overlap_area(grayimg,relative_masks)
            if max_score > acceptable_score:
                print('max score is  {}'.format(max_score))
                    
        if modelName not in available_masks or max_score < acceptable_score:
#             if modelName not in available_masks or max_score < max_diff:
#             print('llllllllllll')
#             print(max_score)
#             print('maximum score is {}'.format(max_score))

            width_img=img.shape[-2] if cm else img.shape[-1]
#  ---------------important------------         
            
              # add help to image for different keys

#             img_hz=np.zeros((size_h[0],width_img,3))
#             img_hz[:,0:size_h[1],:]=help_img
#             #img_hz=img_hz[:, :, ::-1]
# #             cv2.imshow('ffff',img_hz)
#             if cm:
#             #     img[width_img:width_img+size_h[0],0:size_h[1],:]=help_img
#                 img=np.vstack((img,img_hz))
#             else: img=np.vstack((img,cv2.cvtColor(img_hz, cv2.COLOR_BGR2GRAY)))
            
#         -----------------------------------
            cv2.namedWindow('image')
            cv2.setMouseCallback('image',click_save)
            color = (0, 0, 0)
            thickness=2
            vert_help=0
            while(1):
                cv2.imshow('image',img)
                if len(refPt)>3 and vert_help==1:
                    (x1,y1),(x3,y3)=refPt[0],refPt[2]
                    lower=int(min(y1,y3))
                    f=int((x1+x3)/2)
                    cv2.line(img,(f,lower),(f,width_img),(255),thickness)
                    vert_help=0
                k = cv2.waitKey(1) & 0xFF
                if k==ord('h') and feats_rec[3] > -1:
                    cv2.line(img,(feats_rec[3],feats_rec[6]),(feats_rec[4],feats_rec[6]),(255),1)
                    cv2.line(img,(feats_rec[3],feats_rec[5]),(feats_rec[4],feats_rec[5]),(255),1)
                    vert_help=1
                if k == ord('r'):
                    refPt=[]
                if k == 27 and len(refPt) != 5:
                    break
                if k == ord('q'):
                    q=1
                    break
                if k == 27 and len(refPt) == 5:
                    print('1111111111111111111111111111111')
                    poly=create_contour(im_tst,refPt,thickness,size_dial,size_erod)
                    plt.figure()
                    plt.imshow(poly)
                    mask=create_mask(poly,DILATE_ITER,ERODE_ITER,thickness)                    
                    plt.imsave(mask_fpath,mask,cmap="gray")
        #                     keep created mask in the Collections
                    available_masks[modelName].append(mask)
                    available_folder.append(folder_name)
                    available_model.append(modelName)
#                     overlay=apply_mask(im_tst,mask,cm)
#                     plt.imsave(overlay_fpath,overlay)
    #                 plt.figure()
    #                 plt.title('Mask')        
    #                 plt.imshow(mask,cmap=plt.cm.bone)
    #                 plt.figure()
    #                 plt.title('Overlay image') 
    #                 plt.imshow(overlay,cmap=plt.cm.bone)
    #                 cv2.waitKey(0)
            #         show_result(mask,overlay)
            #         plt.imshow(overlay)
                    break
        else: print('maximum score is {}'.format(max_score))
            
df = pd.DataFrame(process_path) 
df.to_csv('processes_records.csv', index=False)

cv2.waitKey(1)
cv2.destroyAllWindows()
cv2.waitKey(1)
plt.close('all')